In [1]:
using Pkg
Pkg.activate(".")
using MLStyle

flatmap(f,v::Vector; init) = foldr(vcat, map(f,v), init=init);

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/CategoryTheory`


In [2]:
struct Leaf end
struct Node{a}
    _1::a
    _2::Vector{Union{Leaf, Node{a}}} # Forest{a} = Vector{Tree{a}} = Vector{Union{Leaf, Node{a}}}
end
Tree{a} = Union{Leaf, Node{a}}
Forest{a} = Vector{Tree{a}};

Forest(n::Node{a}) where a = Tree{a}[n]
zero(::Forest{a}) where a = Tree{a}[]
one(::Forest{a}) where a  = Tree{a}[Leaf()]
⊕(x::Forest, y::Forest)   = vcat(x,y)
⊗(x::Forest, y::Forest)   = begin
    g(::Leaf) = y
    g(n::Node{a}) where a = Tree{a}[Node(n._1, n._2 ⊗ y)]
    flatmap(g, x, init=zero(x))
end

inj(x::T) where T = Forest(Node{T}(x, one(Forest{T}())))
univ(h::Function, xs::Forest) = begin
    univT(::Leaf)  = one(xs)
    univT(n::Node) = h(n._1) ⊕ univ(h, n._2)
    foldr(⊕, map(univT(xs)))
    end

univ (generic function with 1 method)

In [4]:
Forest{Int}()

Union{Leaf, Node{Int64}}[]

In [3]:
inj(10.0)
inj(10.0)

1-element Vector{Union{Leaf, Node{Float64}}}:
 Node{Float64}(10.0, Union{Leaf, Node{Float64}}[Leaf()])

In [19]:
@show zero(Forest{Int}())
@show one(Forest{Int}())
@show Node(1,Tree{Int}[Leaf()]);

zero(Forest{Int}()) = Union{Leaf, Node{Int64}}[]
one(Forest{Int}()) = Union{Leaf, Node{Int64}}[Leaf()]
Node(1, Tree{Int}[Leaf()]) = Node{Int64}(1, Union{Leaf, Node{Int64}}[Leaf()])


In [20]:
x = Tree{Int}[Node(3,zero(Forest{Int}())),Node(4,Tree{Int}[Node(10,Tree{Int}[Leaf()])]),Leaf()]
@show zero(Forest{Int}()) ⊕ x == x == x ⊕ zero(Forest{Int}())
@show one(Forest{Int}()) ⊗ x == x;

zero(Forest{Int}()) ⊕ x == x == x ⊕ zero(Forest{Int}()) = true
one(Forest{Int}()) ⊗ x == x = true


In [23]:
x = Tree{Int}[Node(3,zero(Forest{Int}())),Node(4,Tree{Int}[Node(10,Tree{Int}[Leaf()])]),Leaf()]
y = zero(x)
x ⊗ y

2-element Vector{Union{Leaf, Node{Int64}}}:
 Node{Int64}(3, Union{Leaf, Node{Int64}}[])
 Node{Int64}(4, Union{Leaf, Node{Int64}}[Node{Int64}(10, Union{Leaf, Node{Int64}}[])])

In [24]:
y ⊗ x

Union{Leaf, Node{Int64}}[]

In [25]:
Forest(Node{Int}(3,zero(Forest{Int}()))) ⊕ Forest(Node{Int}(1,zero(Forest{Int}())))

2-element Vector{Union{Leaf, Node{Int64}}}:
 Node{Int64}(3, Union{Leaf, Node{Int64}}[])
 Node{Int64}(1, Union{Leaf, Node{Int64}}[])